In [2]:
import pandas as pd

In [3]:
#Exploring the Dataset
mail_data = pd.read_csv('mail_data.csv')

print(mail_data.shape)
mail_data.head()

(5572, 2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
mail_data['Category'].value_counts(normalize=True)

Category
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

In [5]:
#handling missing values with constant value imputation ('')
data = mail_data.where((pd.notnull(mail_data)), '')

In [6]:
#Training and Test Set

# Randomize the dataset
data_randomized = data.sample(frac=1, random_state = 1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [7]:
#printing the first 5 rows
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
training_set['Category'].value_counts(normalize=True)

Category
ham     0.86541
spam    0.13459
Name: proportion, dtype: float64

In [9]:
test_set['Category'].value_counts(normalize=True)

Category
ham     0.868043
spam    0.131957
Name: proportion, dtype: float64

In [10]:
training_set.dropna(subset=['Message'], inplace=True)

In [11]:
#Data Cleaning
#Lower Case and Remove the Punctuation
import string

training_set['Message'] = training_set['Message'].apply(lambda x: ''.join([char for char in x if char not in string.punctuation]))
training_set['Message'] = training_set['Message'].str.lower()

training_set.head(3)

,Category,Message
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired


In [12]:
training_set['Message'] = training_set['Message'].str.split()

vocabulary = []
for message in training_set['Message']:
   for word in message:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [13]:
len(vocabulary)

8510

In [14]:
word_counts_per_message = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }

word_counts = pd.DataFrame(word_counts_per_message)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [15]:
word_counts_per_message = {unique_word: [0] * len(training_set['Message']) for unique_word in vocabulary}

for index, message in enumerate(training_set['Message']):
   for word in message:
      word_counts_per_message[word][index] += 1

In [18]:
word_counts = pd.DataFrame(word_counts_per_message)
word_counts.head()

,lock,whatever,impatient,placed,responding,m8s,duffer,lololo,suntec,smart,...,theirs,concern,dont,priya,party,suitemates,month,perumbavoor,given,functions
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Category,Message,lock,whatever,impatient,placed,responding,m8s,duffer,lololo,...,theirs,concern,dont,priya,party,suitemates,month,perumbavoor,given,functions
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, theres, a, c...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Category'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Category'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['Message'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['Message'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [21]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

In [22]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')
   else:
      print('Equal proabilities, have a human classify this!')

In [23]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3725376755783706e-25
P(Ham|message): 2.237077346218218e-27
Label: Spam


In [24]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 1.6655594205723071e-25
P(Ham|message): 3.1144397884117738e-21
Label: Ham


In [25]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

In [27]:
test_set['predicted'] = test_set['Message'].apply(classify_test_set)
test_set.head()

,Category,Message,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [28]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Category'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1089
Incorrect: 25
Accuracy: 0.9775583482944344
